In [50]:
import json
import os
import time
from selenium import webdriver
from math import pi, tan, inf
import numpy as np
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shortest_path_calc import *
import matplotlib as mpl
import branca.colormap as cm
import imageio
import glob
import networkx.algorithms.approximation as nx_app

In [15]:
#Flight height in m
flight_height = 60.0 #ATH þessi verður input

#Sensor width in inches
sensor_width = 1/2.4

#Converting sensor with to mm
sensor_width *= 25.4

#Focal length in mm
focal_length = 24 # ATH þurfum að finna þessa tölu betur

#Angle of view in degrees
angle_of_view = 2*np.arctan(sensor_width / (2*focal_length)) * (180/pi)

#Field of view in m
field_of_view = abs(2*(tan(angle_of_view/2)*flight_height))


print("field of view: ", field_of_view)

f = open('data.json')
#f = open('test_data_lyngby.json')
data = json.load(f)
x = []
y = []


for d in data:
    x.append(d['x'])
    y.append(d['y'])


max_x = max(x)
min_x = min(x)
max_y = max(y)
min_y = min(y)

x_cur = min_x
y_cur = min_y

lat_fov, lon_fov = calc_field_of_view_GPS(flight_height=flight_height, first_lat=data[0]['x'])

grid_coordinates = []


new_tuple = [(x[idx],y[idx]) for idx in range(len(x))]
polygon = Polygon(new_tuple)
first = True
while(x_cur <= max_x):
    y_cur = min_y
    while(y_cur <= max_y):
        point2 = Point(x_cur ,y_cur)
        if polygon.contains(point2):
            grid_coordinates.append({"x":x_cur,"y":y_cur})
        y_cur += lon_fov
    x_cur += lat_fov

#colormap = cm.LinearColormap(colors=['red', 'blue','green'], index=[0, len(grid_coordinates)], vmin=0, vmax=len(grid_coordinates))
#map = folium.Map(location=[55.78335044630297,12.494188681035387], zoom_start=100)
map = folium.Map(location=[55.65031181498796, 12.570402704029256], zoom_start=16)

for i, point in enumerate(grid_coordinates):
    folium.CircleMarker([point["x"], point["y"]], radius=1,color="red").add_to(map)
    
map





field of view:  15.980087863726574


In [52]:
import geopy.distance as geo_dist
start = (55.64809738877586, 12.569831107601916)
min_dist = inf
for node in G.nodes:
    dist = geo_dist.geodesic(start, (G.nodes[node]['coordinate']['x'],G.nodes[node]['coordinate']['y'])).m
    if dist < min_dist:
        min_dist = dist
        min_node = node
    
print(min_dist)
print(node)

0.0
45


In [53]:
#Creates graph and calculates shortest path

G = nx.Graph()

for idx, coordinate in enumerate(grid_coordinates):
    G.add_node(idx, is_full=0, coordinate=coordinate)

max_dist = np.sqrt(2*(field_of_view**2))
G2 = G.copy()
for node in G:
    G2.remove_node(node)
    for node2 in G2:
        x_dist = abs(G.nodes[node]['coordinate']['x'] - G2.nodes[node2]['coordinate']['x'])
        y_dist = abs(G.nodes[node]['coordinate']['y'] - G2.nodes[node2]['coordinate']['y'])
        if (x_dist <= max_dist and y_dist <= max_dist):
            G.add_edge(node, node2, weight = np.sqrt(x_dist**2 + y_dist**2))


#path = nx_app.traveling_salesman_problem(G, cycle=True)
path = nx_app.greedy_tsp(G,source=node)

list_of_coordinates = []
for node in path:
    list_of_coordinates.append(G.nodes[node]['coordinate'])

In [54]:
#Creates html and png files for gif
map = folium.Map(location=[55.65031181498796, 12.570402704029256], zoom_start=15)

for i, point in enumerate(list_of_coordinates):
    folium.CircleMarker([point["x"], point["y"]], radius=1,color="red").add_to(map)
    map.save("./html_files/Drone_map_"+str(i)+".html")

path = "/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux"
delay=5
os.chdir(path)

for i, point in enumerate(list_of_coordinates):
    fn="./html_files/Drone_map_"+str(i)+".html"
    tmpurl = 'file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    
    browser = webdriver.Chrome()
    browser.get(tmpurl)
    
    time.sleep(delay)
    browser.save_screenshot("./html_files/Drone_map_"+str(i)+".png")
    browser.quit()
    
    os.remove("./html_files/Drone_map_"+str(i)+".html")

In [55]:
#Creates gif
image_path = os.path.join(path, "html_files")

image_list = []

list_of_files = filter(os.path.isfile, glob.glob(path + "/html_files/" + "*"))
list_of_files = sorted(list_of_files, key=os.path.getmtime)

for file_name in list_of_files:
    if not file_name.startswith("."):
        print(file_name)
        image_list.append(imageio.imread(os.path.join("html_files",file_name)))
        #os.remove(file_name)

/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_0.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_1.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_2.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_3.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_4.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_5.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_6.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_7.png
/Users/hildurmargret/Documents/DTU/Haust 2022/Advanced Project/Drone-Project-Dalux/html_files/Drone_map_8.png
/Users/hil

In [56]:
#Create the gif
imageio.mimwrite('GifMap.gif', image_list, fps=3)

In [54]:
data[0]['x']

55.652947165081294